In [12]:
# from nltk import ngrams

# sentence = 'this is a foo bar sentences and i want to ngramize it'

# n = 6
# sixgrams = ngrams(sentence.split(), n)

# for grams in sixgrams:
#   print(grams)

In [13]:
import csv
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

def read_data(path):
    data=[]
    with open(path, 'rt') as f:
        reader = csv.reader(f, delimiter=';')
        for row in reader:
            row[0] = row[0].replace(',', '')
            str=row[0].split()
            str[0]=str[0][1:]
            str[len(str)-1]=str[len(str)-1][:-1]
            str=[i[1:-1] for i in str] 
            data.append(' '.join(str))
        return data


############### PATHS
############### STOPWORDS INCLUDED
test_pos=read_data("../assignment_1/pos/test.csv")
val_pos=read_data("../assignment_1/pos/val.csv")
train_pos=read_data("../assignment_1/pos/train.csv")

test_neg=read_data("../assignment_1/neg/test.csv")
val_neg=read_data("../assignment_1/neg/val.csv")
train_neg=read_data("../assignment_1/neg/train.csv")


################# STOPWORDS REMOVED
test_pos_no_stopwords=read_data("../assignment_1/pos/test_no_stopword.csv")
val_pos_no_stopwords=read_data("../assignment_1/pos/val_no_stopword.csv")
train_pos_no_stopwords=read_data("../assignment_1/pos/train_no_stopword.csv")

test_neg_no_stopwords=read_data("../assignment_1/neg/test_no_stopword.csv")
val_neg_no_stopwords=read_data("../assignment_1/neg/val_no_stopword.csv")
train_neg_no_stopwords=read_data("../assignment_1/neg/train_no_stopword.csv")

############


def shuffle(data_pos,data_neg): 
    data = data_pos + data_neg
    merged_target = [0.0] * len(data_pos) +[1.0] * len(data_neg)
    d={'data':data,'target':merged_target}
    df = pd.DataFrame(d, columns=['data','target'])
    df = df.sample(frac=1).reset_index(drop=True)
    return list(df['data']), np.array(list(df['target']))
#############

print("complete")

complete


In [14]:
def find_test_accuracy(stopwords_include, grams):
    if stopwords_include == 'yes':
        train, train_target = shuffle(train_pos, train_neg)
        val, val_target = shuffle(val_pos, val_neg)
        test, test_target = shuffle(test_pos, test_neg)
        # train, train_target=train[0:1], train_target[0:1]
        print(train[0])
    elif stopwords_include == 'no':
        train, train_target = shuffle(train_pos_no_stopwords, train_neg_no_stopwords)
        val, val_target = shuffle(val_pos_no_stopwords, val_neg_no_stopwords)
        test, test_target = shuffle(test_pos_no_stopwords, test_neg_no_stopwords)
        # train, train_target=train[0:1], train_target[0:1]
    if grams == 'unigrams':
        vector = CountVectorizer(stop_words=[])# only unigrams
    elif grams == 'bigrams':
        vector = CountVectorizer(ngram_range=(2,2),stop_words=[])# unigrams + bigrams
    else:
        vector = CountVectorizer(ngram_range=(1,2),stop_words=[])# only bigrams
    vector.fit(train)
    counts = vector.transform(train)
    print(vector.vocabulary_)
    # print(counts.toarray())
    # print(counts.shape)
    tfidf_transformer = TfidfTransformer()
    xtrain = tfidf_transformer.fit_transform(counts)
    print(xtrain.shape)
    
    
    # HYPERPARAMETER TUNING USING VALIDATION SET
    print('tuning hyperparameters...')
    max_acc=-1
    x_max=-1
    for x in np.arange(0.1,10.0,0.2):
        clf = MultinomialNB(alpha = x, class_prior=None, fit_prior=True).fit(xtrain,train_target)
        x_new_counts = vector.transform(val)
        x_new_tfidf = tfidf_transformer.transform(x_new_counts)
        predicted = clf.predict(x_new_tfidf)
        acc = accuracy_score(val_target, predicted)
        print(acc)
        if max_acc < acc:
            max_acc = acc
            x_max = x    
    
    # TEST ACCURACY
    clf = MultinomialNB(alpha = x_max, class_prior=None, fit_prior=True).fit(xtrain,train_target)
    x_new_counts = vector.transform(test)
    x_new_tfidf = tfidf_transformer.transform(x_new_counts)
    predicted = clf.predict(x_new_tfidf)
    acc = accuracy_score(test_target, predicted)
    print('best alpha is', x_max)
    print('accuracy is',acc)
    

In [ ]:
print("STOPWORDS_REMOVED = NO, TEXT_FEATURES= UNIGRAMS")
find_test_accuracy('no','unigrams')

print("STOPWORDS_REMOVED = NO, TEXT_FEATURES= BIGRAMS")
find_test_accuracy('no','bigrams')

print("STOPWORDS_REMOVED = NO, TEXT_FEATURES= UNIGRAMS + BIGRAMS")
find_test_accuracy('no','both')

print("STOPWORDS_REMOVED = YES, TEXT_FEATURES= UNIGRAMS")
find_test_accuracy('yes','unigrams')

print("STOPWORDS_REMOVED = YES, TEXT_FEATURES= BIGRAMS")
find_test_accuracy('yes','bigrams')

print("STOPWORDS_REMOVED = YES, TEXT_FEATURES= UNIGRAMS + BIGRAMS")
find_test_accuracy('yes','both')


STOPWORDS_REMOVED = NO, TEXT_FEATURES= UNIGRAMS
{'scale': 56801, '10': 181, 'would': 72958, 'give': 29485, '11': 420, 'looking': 39377, 'replace': 54348, 'old': 45683, 'dull': 22771, 'pizza': 49246, 'wheel': 71942, 'discovered': 21390, 'one': 45792, 'ordered': 46168, 'case': 13755, 'ipad': 35427, 'expecting': 25361, 'nice': 44372, 'alternative': 6350, 'smart': 59670, 'cover': 17963, 'always': 6403, 'gets': 29295, 'job': 36287, 'done': 22106, 'quickly': 52342, 'easily': 23235, 'thing': 65670, 'weighs': 71676, 'like': 38747, 'pound': 50269, 'way': 71472, 'heavy': 31941, 'tire': 66300, 'tread': 67234, 'gauge': 29025, 'got': 29988, 'two': 67983, 'similiar': 58880, 'beads': 9596, 'package': 47162, 'sliighty': 59455, 'different': 20931, 'shade': 57983, 'purple': 51958, 'metal': 41562, 'right': 55267, 'balance': 9028, 'flex': 27150, 'yet': 73552, 'stiff': 62029, 'enough': 24276, 'hold': 32543, 'meats': 41163, 'without': 72587, 'bending': 10006, 'solve': 60275, 'problem': 51051, 'review': 5500

0.723375
0.72615
0.7276125
0.7284875
0.7291125
0.7298125
0.7299875
0.7302375
0.730575
0.730925
0.7310625
0.7311125
0.7311
0.731275
0.73125
0.7313
0.731375
0.7312
0.7311875
0.731275
0.731275
0.7313125
0.7312875
0.731325
0.731575
0.731575
0.7316375
0.731625
0.7316125
0.7317875
0.73185
0.7319
0.732
0.7319875
0.7320625
0.732
0.731925
0.7319875
0.732075
0.73215
0.732275
0.7323375
0.7324125
0.732475
0.7325
0.73265
0.73255
0.7326375
0.732725
0.7328
best alpha is 9.900000000000002
accuracy is 0.7307875
STOPWORDS_REMOVED = NO, TEXT_FEATURES= BIGRAMS


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(640000, 1839255)
tuning hyperparameters...
0.703175
0.711075
0.713975
0.7155
0.7161625
0.7166625
0.717025
0.71695
0.71705
0.7167125
0.716425
0.7164
0.7162125
0.71615
0.716075
0.7159875
0.7157875
0.7156625
0.7154375
0.715475
0.7151
0.7148375
0.7146
0.7144375
0.71415
0.7139125
0.7136625
0.713525
0.713325
0.7132125
0.7130125
0.71285
0.7126625
0.71245
0.7121625
0.712075
0.71185
0.711625
0.7116
0.7114875
0.7113375
0.7112
0.710925
0.7109125
0.7106375
0.7105125
0.7104125
0.71015
0.71015
0.709975
best alpha is 1.7000000000000004
accuracy is 0.673075
STOPWORDS_REMOVED = NO, TEXT_FEATURES= UNIGRAMS + BIGRAMS


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(640000, 1913329)
tuning hyperparameters...
0.7533
0.761325
0.76325
0.7643125
0.7650125
0.7649875
0.7646625
0.764425
0.763825
0.7640125
0.7638375
0.7635625
0.7631625
0.7631
0.7627125
0.762325
0.76195
0.7614875
0.7611375
0.76085
0.7605
0.7602125
0.7600875
0.759925
0.7596875
0.75955
0.7592125
0.759075
0.7587
0.7584125
0.758375
0.75825
0.7580875
0.7579875
0.7577375
0.7575125
0.757325
0.757175
0.756925
0.756825
0.7567
0.75655
0.75655
0.756325
0.756075
0.7560125
0.755725
0.7556875
0.755475
0.7552125
best alpha is 0.9000000000000001
accuracy is 0.755625
STOPWORDS_REMOVED = YES, TEXT_FEATURES= UNIGRAMS
in addition to the variety of wearing options jabra has included both wall and car chargers for the extreme .
{'in': 34126, 'addition': 5275, 'to': 66513, 'the': 65178, 'variety': 70163, 'of': 45561, 'wearing': 71669, 'options': 46183, 'jabra': 35983, 'has': 31564, 'included': 34254, 'both': 11412, 'wall': 71224, 'and': 6709, 'car': 13474, 'chargers': 14451, 'for': 27682, 'extreme': 25642, 'am'

(640000, 74217)
tuning hyperparameters...
0.727475
0.730225
0.7314125
0.7323375
0.7326875
0.732925
0.733475
0.7338125
0.7342125
0.7346
0.7347625
0.7347625
0.7348375
0.73485
0.73495
0.7349125
0.7349875
0.7351125
0.735225
0.735125
0.7354
0.73555
0.73555
0.7353375
0.73515
0.73535
0.73535
0.7354125
0.735525
0.7354875
0.73535
0.73535
0.735425
0.7355875
0.7354375
0.735525
0.7355625
0.7355
0.7355
0.7354375
0.735525
0.7356
0.7355
0.7356
0.7355875
0.735575
0.735525
0.7354125
0.7353625
0.7354375
best alpha is 8.3
accuracy is 0.7314625
STOPWORDS_REMOVED = YES, TEXT_FEATURES= BIGRAMS
i would not use anything else . . . and hey they are pyrex .


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(640000, 1394997)
tuning hyperparameters...
0.763
0.76975
0.7710875
0.7723625
0.77215
0.7715625
0.771125
0.7707375
0.76995
0.769025
0.76875
0.7684875
0.76835
